# Lesen und Visualisieren von Rohdaten, Zuschneiden, Filtern und Speichern

## Importiere die wenigen Pakete, die ich brauche

In [24]:
import matplotlib
import pathlib
import mne

Ensure Matplotlib uses the `Qt5Agg` backend, which is the best choice for MNE-Python's interactive plotting functions.

In [2]:
matplotlib.use('Qt5Agg')

In [25]:
sample_data_dir = mne.datasets.sample.data_path()

# Für mehr Komfort in einen pathlib.Path konvertieren
sample_data_dir = pathlib.Path(sample_data_dir)
sample_data_dir

WindowsPath('C:/Users/KARMERY/mne_data/MNE-sample-data')

## Noch einige Rohdaten laden.

In [ ]:
raw_path = sample_data_dir / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
raw = mne.io.read_raw(raw_path)
raw

## Visualisierung der Rohdaten!

In [6]:
raw.plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 800x800 with 5 Axes>

## Ereignisse aus den `STIM`-Kanälen extrahieren

In [7]:
events = mne.find_events(raw)

320 events found
Event IDs: [ 1  2  3  4  5 32]


In [27]:
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}
event_id

{'Auditory/Left': 1,
 'Auditory/Right': 2,
 'Visual/Left': 3,
 'Visual/Right': 4,
 'Smiley': 5,
 'Button': 32}

In [8]:
len(events[events[:, 2] == 32])

16

## Die Rohdaten erneut zeichnen, aber die Ereignismarkierungen hinzufügen.

In [28]:
raw.plot(events=events, event_id=event_id)

<MNEBrowseFigure size 800x800 with 5 Axes>

## Informationen über den Daten Sammeln

In [29]:
raw.info

<Info | 21 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: 4 items (MEG 2443, EEG 053, EEG 051, EEG 050)
 ch_names: MEG 0113, MEG 0112, MEG 0111, MEG 0122, MEG 0123, MEG 0121, MEG ...
 chs: 204 Gradiometers, 102 Magnetometers, 9 Stimulus, 60 EEG, 1 EOG
 custom_ref_applied: False
 description: acquisition (megacq) VectorView system at NMR-MGH
 dev_head_t: MEG device -> head transform
 dig: 146 items (3 Cardinal, 4 HPI, 61 EEG, 78 Extra)
 events: 1 item (list)
 experimenter: MEG
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 lowpass: 172.2 Hz
 meas_date: 2002-12-03 19:01:10 UTC
 meas_id: 4 items (dict)
 nchan: 376
 proj_id: 1 item (ndarray)
 proj_name: test
 projs: PCA-v1: off, PCA-v2: off, PCA-v3: off
 sfreq: 600.6 Hz
>

In [30]:
raw.info['meas_date']

datetime.datetime(2002, 12, 3, 19, 1, 10, 720100, tzinfo=datetime.timezone.utc)

In [31]:
raw.info['sfreq']

600.614990234375

In [32]:
raw.info['bads']

['MEG 2443', 'EEG 053', 'EEG 051', 'EEG 050']

In [33]:
raw.ch_names[:10]

['MEG 0113',
 'MEG 0112',
 'MEG 0111',
 'MEG 0122',
 'MEG 0123',
 'MEG 0121',
 'MEG 0132',
 'MEG 0133',
 'MEG 0131',
 'MEG 0143']

In [34]:
raw.info['chs'][0]

{'scanno': 1,
 'logno': 113,
 'kind': 1 (FIFFV_MEG_CH),
 'range': 0.00030517578125,
 'cal': 3.1600000394149674e-09,
 'coil_type': 3012 (FIFFV_COIL_VV_PLANAR_T1),
 'loc': array([-0.1066    ,  0.0464    , -0.0604    , -0.0127    ,  0.0057    ,
        -0.99990302, -0.186801  , -0.98240298, -0.0033    , -0.98232698,
         0.18674099,  0.013541  ]),
 'unit': 201 (FIFF_UNIT_T_M),
 'unit_mul': 0 (FIFF_UNITM_NONE),
 'ch_name': 'MEG 0113',
 'coord_frame': 1 (FIFFV_COORD_DEVICE)}

## Visualisierung die Sensorpositionen 

In [35]:
raw.plot_sensors(ch_type='eeg')

<Figure size 640x640 with 1 Axes>

In [37]:
raw.plot_sensors(kind='3d', ch_type='eeg')

<Figure size 640x640 with 1 Axes>

## Kanäle als schlecht markieren

Ich markiere einen zusätzlichen EEG-Kanal als schlecht und sehe mich das Topoplot an.

In [38]:
raw.info['bads']

['MEG 2443', 'EEG 053', 'EEG 051', 'EEG 050']

In [39]:
raw.info['bads'] += ['EEG 051', 'EEG 050']
raw.plot_sensors(ch_type='eeg')

<Figure size 640x640 with 1 Axes>

## Nur eine Teilmenge der Kanäle auswählen.

In [40]:
raw_eeg = raw.copy().pick_types(meg=False, eeg=True, eog=True, exclude=[])
len(raw_eeg.ch_names)

Removing projector <Projection | PCA-v1, active : False, n_channels : 102>
Removing projector <Projection | PCA-v2, active : False, n_channels : 102>
Removing projector <Projection | PCA-v3, active : False, n_channels : 102>


61

In [41]:
raw_eeg.info

<Info | 20 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: 5 items (EEG 053, EEG 051, EEG 050, EEG 051, EEG 050)
 ch_names: EEG 001, EEG 002, EEG 003, EEG 004, EEG 005, EEG 006, EEG 007, ...
 chs: 60 EEG, 1 EOG
 custom_ref_applied: False
 description: acquisition (megacq) VectorView system at NMR-MGH
 dev_head_t: MEG device -> head transform
 dig: 146 items (3 Cardinal, 4 HPI, 61 EEG, 78 Extra)
 events: 1 item (list)
 experimenter: MEG
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 lowpass: 172.2 Hz
 meas_date: 2002-12-03 19:01:10 UTC
 meas_id: 4 items (dict)
 nchan: 61
 proj_id: 1 item (ndarray)
 proj_name: test
 projs: []
 sfreq: 600.6 Hz
>

In [42]:
raw_eeg.plot(events=events, event_id=event_id)

<MNEBrowseFigure size 800x800 with 4 Axes>

Channels marked as bad:
['EEG 053', 'EEG 051', 'EEG 050', 'EEG 051', 'EEG 050']


## Crop die Daten und filtern.

In [43]:
raw_eeg_cropped = raw_eeg.copy().crop(tmax=100)
raw_eeg_cropped.times[-1]

99.99916914588277

In [45]:
raw_eeg_cropped.load_data()
raw_eeg_cropped_filtered = raw_eeg_cropped.copy().filter(l_freq=0.1, h_freq=40)

Reading 0 ... 60061  =      0.000 ...    99.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 19821 samples (33.001 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    1.1s finished


In [46]:
raw_eeg_cropped.plot(events=events, event_id=event_id, title='Unfiltered' )
raw_eeg_cropped_filtered.plot(events=events, event_id=event_id, title='Filtered')

<MNEBrowseFigure size 800x800 with 4 Axes>

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)

raw_eeg_cropped.plot_psd(ax=ax[0], show=False)
raw_eeg_cropped_filtered.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD vor dem Filtern')
ax[1].set_title('PSD nach dem Filtern')
ax[1].set_xlabel('Frequenz (Hz)')
fig.set_tight_layout(True)
plt.show()

## Speichern die Daten!

In [23]:
raw_eeg_cropped_filtered.save(pathlib.Path('out_data') / 'eeg_cropped_filt_raw.fif', 
                              overwrite=True)

NameError: name 'raw_eeg_cropped_filtered' is not defined